In [0]:
from numpy import expand_dims, ones, zeros, vstack, array, mean
from numpy.random import randn, randint, normal
from keras.optimizers import Adam
from keras.models import Sequential, Model
import tensorflow as tf
from keras.layers import Dense, Reshape, Flatten, Conv2D, Input, Lambda, concatenate, UpSampling2D
from keras.layers import Conv2DTranspose, LeakyReLU, Dropout, GaussianNoise, BatchNormalization, AveragePooling2D
from keras.initializers import RandomNormal
from keras.utils import plot_model
from keras.datasets.cifar10 import load_data
from matplotlib import pyplot
import glob
import cv2
import keras.backend as K
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

# define the standalone discriminator model
def define_discriminator(in_shape=(128,128,3)):
  init = RandomNormal(mean=0.0, stddev=0.02)
  model = Sequential()
  # 128x128
  main_input = Input(shape=in_shape)
  model.add(Conv2D(64, (5,5), padding='same', input_shape=in_shape, kernel_initializer=init))
  model.add(BatchNormalization(momentum=0.8))
  model.add(LeakyReLU(alpha=0.2))
  model.add(AveragePooling2D()) # downsample to 64x64

  model.add(Conv2D(128, (5,5), padding='same', kernel_initializer=init))
  model.add(BatchNormalization(momentum=0.8))
  model.add(LeakyReLU(alpha=0.2))
  model.add(AveragePooling2D()) # downsample 32x32

  model.add(Conv2D(256, (5,5), padding='same', kernel_initializer=init))
  model.add(BatchNormalization(momentum=0.8))
  model.add(LeakyReLU(alpha=0.2))
  model.add(AveragePooling2D()) # downsample 16x16

  model.add(Conv2D(512, (5,5), padding='same', kernel_initializer=init))
  model.add(BatchNormalization(momentum=0.8))
  model.add(LeakyReLU(alpha=0.2))
  model.add(AveragePooling2D()) # downsample 8x8
  # dense classifier
  model.add(Flatten())
  model.add(Dense(1024))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dense(512))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dense(128))
  model.add(LeakyReLU(alpha=0.2))
  # output
  model.add(Dense(1, activation='sigmoid'))
  # compile model
  opt = Adam(lr=0.0001, beta_1=0.05) # low learning rate
  bce = tf.keras.losses.BinaryCrossentropy(from_logits=False)
  model.compile(loss=bce, optimizer=opt, metrics=['accuracy'])
  return model

# define the standalone generator model
def define_generator(latent_dim):
	model = Sequential()
	init = RandomNormal(mean=0.0, stddev=0.02)
	# foundation for 16x16 image
	model.add(Dense(64, input_dim=latent_dim))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Dense(512, input_dim=latent_dim))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Dense(16384, input_dim=latent_dim))
	model.add(BatchNormalization(momentum=0.8))
	model.add(Reshape((16, 16, 64)))
	model.add(BatchNormalization(momentum=0.8))
    # upsample
	model.add(Conv2D(256, (5,5), padding='same', kernel_initializer=init))
	model.add(BatchNormalization(momentum=0.8))
	model.add(LeakyReLU(alpha=0.2))
	model.add(BatchNormalization(momentum=0.8))
	model.add(UpSampling2D())
	# upsample
	model.add(Conv2D(128, (5,5), padding='same', kernel_initializer=init))
	model.add(BatchNormalization(momentum=0.8))
	model.add(LeakyReLU(alpha=0.2))
	model.add(UpSampling2D())
	# upsample
	model.add(Conv2D(64, (5,5), padding='same', kernel_initializer=init))
	model.add(BatchNormalization(momentum=0.8))
	model.add(LeakyReLU(alpha=0.2))
	model.add(UpSampling2D())
	# output layer
	model.add(Conv2D(3, (5,5), activation='tanh', padding='same', kernel_initializer=init))
	return model


# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(g_model)
	# add the discriminator
	model.add(d_model)
	# compile model
	opt = Adam(lr=0.0001, beta_1=0.05)
	bce = tf.keras.losses.BinaryCrossentropy(from_logits=False)
	model.compile(loss=bce, optimizer=opt)
	return model

# load and prepare cifar10 training images
def load_real_samples():
	trainX = []
	files = glob.glob(F'/content/gdrive/My Drive/projeto_machine_learning/resized_128x128/*.jpg')
	i=1
	for myFile in files:
		image = cv2.imread(myFile)
		print('%04d'%i +'read '+myFile )
		trainX.append(image)
		i=i+1
	X = array(trainX,dtype='float32')
	# scale from [0,255] to [-1,1]
	X = (X - 127.5) / 127.5
	return X

# load and prepare cifar10 training images (unused here)
def load_real_samples2():
	# load cifar10 dataset
	(trainX, _), (_, _) = load_data()
	# convert from unsigned ints to floats
	X = trainX.astype('float32')
	# scale from [0,255] to [-1,1]
	X = (X - 127.5) / 127.5
	return X

# select real samples
def generate_real_samples(dataset, n_samples, var):
	# choose random instances
	ix = randint(0, dataset.shape[0], n_samples)
	# retrieve selected images
	X = dataset[ix]
	# instantiate noise
	noise = normal(0,var,X.shape)
	# add noise
	X = X + noise
	# generate 'real' class labels (1)
	y = ones((n_samples, 1))
	return X, y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples, var):
	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	X = g_model.predict(x_input)
 	# instantiate noise
	noise = normal(0,var,X.shape)
	# add noise
	X = X + noise
	# create 'fake' class labels (0)
	y = zeros((n_samples, 1))
	return X, y

# create and save a plot of generated images
def save_plot(examples, epoch, n=7):
	# scale from [-1,1] to [0,1]
	examples = (examples + 1) / 2.0
	# plot images
	for i in range(n * n):
		# define subplot
		pyplot.subplot(n, n, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(examples[i])
	# save plot to file
	drivePath = "/content/gdrive/My Drive/projeto_machine_learning/paint_files/"
	filename = 'generated_plot_e%04d.png' % (epoch+1)
	pyplot.savefig(drivePath+filename)
	pyplot.close()

# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=150):
	# prepare real samples
	X_real, y_real = generate_real_samples(dataset, n_samples, 0)
	# evaluate discriminator on real examples
	_, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
	# prepare fake examples
	x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples, 0)
	# evaluate discriminator on fake examples
	_, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
	# summarize discriminator performance
	print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
	drivePath = "/content/gdrive/My Drive/projeto_machine_learning/paint_files/"
	# save plot
	save_plot(x_fake, epoch)
	# save the generator model tile file
	filename1 = 'generated_plot_%04d.png' % (epoch+1)
	filename2 = 'generator_model_%03d.h5' % (epoch+1)
	g_model.save(drivePath+filename2)
	print('>Saved: %s and %s' % (filename1, filename2))
	# save the generator model
	filename2 = 'model_%04d.h5' % (epoch+1)
	g_model.save(drivePath+filename2)
	print('>Saved: %s and %s' % (filename1, filename2))

def print_acc(epoch, g_model, d_model, dataset, latent_dim, n_samples=150):
	# prepare real samples
	X_real, y_real = generate_real_samples(dataset, n_samples, 0)
	# evaluate discriminator on real examples
	_, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
	# prepare fake examples
	x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples, 0)
	# evaluate discriminator on fake examples
	_, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
	# summarize discriminator performance
	print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))

# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=1000, n_batch=16, n_critic=1, noise_pwr=0.01):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
	# manually enumerate epochs
    for i in range(n_epochs):
    # enumerate batches over the training set
        for j in range(bat_per_epo):
            c1_tmp, c2_tmp = list(), list()
            for _ in range(n_critic):
                # get randomly selected 'real' samples
                X_real, y_real = generate_real_samples(dataset, half_batch, 1.1*noise_pwr-noise_pwr*(i/n_epochs))
                # update critic model weights
                c_loss1 = d_model.train_on_batch(X_real, y_real)
                c1_tmp.append(c_loss1)
                # generate 'fake' examples
                X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch, 1.1*noise_pwr-noise_pwr*(i/n_epochs))
                # update critic model weights
                c_loss2 = d_model.train_on_batch(X_fake, y_fake)
                c2_tmp.append(c_loss2)
            # store critic loss
            d_loss1 = mean(c1_tmp)
            d_loss2 = mean(c2_tmp)
			# prepare points in latent space as input for the generator
            X_gan = generate_latent_points(latent_dim, n_batch)
			# create inverted labels for the fake samples
            y_gan = ones((n_batch, 1))
			# update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            # summarize loss on this batch
            print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %(i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
        if (i+1) % 20 == 0:
            summarize_performance(i, g_model, d_model, dataset, latent_dim)
        	

from google.colab import drive
drive.mount("/content/gdrive")

# size of the latent space
latent_dim = 512
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()
# train model
train(g_model, d_model, gan_model, dataset, latent_dim)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
0001read /content/gdrive/My Drive/projeto_machine_learning/resized_128x128/128x128_Alfred_Sisley_1.jpg
0002read /content/gdrive/My Drive/projeto_machine_learning/resized_128x128/128x128_Alfred_Sisley_105.jpg
0003read /content/gdrive/My Drive/projeto_machine_learning/resized_128x128/128x128_Alfred_Sisley_104.jpg
0004read /content/gdrive/My Drive/projeto_machine_learning/resized_128x128/128x128_Alfred_Sisley_103.jpg
0005read /content/gdrive/My Drive/projeto_machine_learning/resized_128x128/128x128_Alfred_Sisley_106.jpg
0006read /content/gdrive/My Drive/projeto_machine_learning/resized_128x128/128x128_Alfred_Sisley_102.jpg
0007read /content/gdrive/My Drive/projeto_machine_learning/resized_128x128/128x128_Alfred_Sisley_100.jpg
0008read /content/gdrive/My Drive/projeto_machine_learning/resized_128x128/128x128_Alfred_Sisley_101.jpg
0009read /content/gdrive/My Dri

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


>1, 1/23, d1=0.590, d2=3.948 g=0.916


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


>1, 2/23, d1=0.585, d2=1.160 g=1.671
>1, 3/23, d1=0.591, d2=0.880 g=1.699
>1, 4/23, d1=0.634, d2=0.818 g=1.945
>1, 5/23, d1=0.599, d2=0.562 g=1.720
>1, 6/23, d1=0.662, d2=0.570 g=2.340
>1, 7/23, d1=0.640, d2=0.665 g=2.573
>1, 8/23, d1=0.687, d2=0.743 g=3.319
>1, 9/23, d1=0.944, d2=0.600 g=2.183
>1, 10/23, d1=0.707, d2=0.653 g=2.142
>1, 11/23, d1=0.555, d2=0.687 g=2.037
>1, 12/23, d1=0.468, d2=0.705 g=2.690
>1, 13/23, d1=0.681, d2=1.145 g=2.318
>1, 14/23, d1=0.621, d2=0.610 g=2.356
>1, 15/23, d1=0.550, d2=0.704 g=2.106
>1, 16/23, d1=0.585, d2=0.713 g=1.894
>1, 17/23, d1=0.744, d2=0.557 g=1.685
>1, 18/23, d1=0.596, d2=0.588 g=1.621
>1, 19/23, d1=0.621, d2=0.703 g=1.657
>1, 20/23, d1=0.577, d2=0.831 g=1.912
>1, 21/23, d1=0.673, d2=0.529 g=1.234
>1, 22/23, d1=0.649, d2=0.625 g=1.210
>1, 23/23, d1=0.537, d2=0.533 g=1.166
>2, 1/23, d1=0.546, d2=0.643 g=1.086
>2, 2/23, d1=0.539, d2=0.640 g=1.436
>2, 3/23, d1=0.637, d2=0.632 g=1.117
>2, 4/23, d1=0.602, d2=0.494 g=1.343
>2, 5/23, d1=0.565, d2=0